# How to save parameters along with the function 

This is a workaround for the problem of how to save *and restore* the parameters of a workflow (e.g., sigma for blurring, threshold values, etc) along with the functions. The idea is simple:

Instead of setting workflow steps with, for instance, `w.set('blurring', blur_function, 'input', sigma=5)`, this notebook creates a partial function of `blur_function()` which is then added to the workflow. When the workflow is loaded and the partial function is passed to magicgui, the parameters are restored correctly.

In [102]:
from napari_workflows import Workflow
from napari.types import ImageData
import napari
import yaml
from skimage.filters import threshold_otsu, gaussian

import inspect
import magicgui
from functools import partial

In [103]:
w = Workflow()

# define segmentation
def threshold(image: ImageData, value: float = 1) -> ImageData:
    return image > value

def add(imageA: ImageData, imageB:ImageData) -> ImageData:
    return imageA + imageB

def blur(image: ImageData, sigma: float=2):
    return gaussian(image, sigma=sigma)

_blur = partial(blur, sigma=5)

w.set('blurring', _blur, 'input', sigma=5)
w.set('binarization', threshold, 'blurring')


with open('workflow.yaml', 'w') as f:
    yaml.dump(w, f)

In [104]:
with open("workflow.yaml", "rb") as stream:
    w=yaml.unsafe_load(stream)

In [105]:
w._tasks

{'binarization': (<function __main__.threshold(image: <function NewType.<locals>.new_type at 0x000002BC1715F9D0>, value: float = 1) -> <function NewType.<locals>.new_type at 0x000002BC1715F9D0>>,
  'blurring',
  1),
 'blurring': (functools.partial(<function blur at 0x000002BC47C70D30>, sigma=5),
  'input',
  5)}

In [111]:
viewer = napari.Viewer()
steps = w._tasks.keys()

['binarization', 'blurring']

In [115]:
viewer = napari.Viewer()

for key in w._tasks.keys():
    func = w._tasks[key][0]
    widget = magicgui.magicgui(func)
    viewer.window.add_dock_widget(widget)

In [ ]:
napari.utils.nbscreenshot(viewer, canvas_only=False)